In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
from txtai.embeddings import Embeddings
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("amazon/MistralLite", model_max_length = 2000)
model = AutoModelForCausalLM.from_pretrained("amazon/MistralLite", pad_token_id = tokenizer.eos_token_id)

/home/jdownes/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.26it/s]


In [3]:
# Create the knowledge base
embeddings = Embeddings(
    {"path": "sentence-transformers/multi-qa-mpnet-base-dot-v1", "content": True, "tokenize": True}
)
chat_data=[]
user1_context=["I hate spicy food.", "My favorite type of foods are seafood."]
user2_context=["I love southern food", "I am open to trying all types of food"]
shared_knowledge=["The best restaurant anywhere is Burger King!","The best restaurant in Chicago is Pequod's Pizza"]
for d in user1_context:
    chat_data.append({"text": d, "user_id":1})
for d in user2_context:
    chat_data.append({"text": d, "user_id":2})

#User 0 is our shared knowledge space
for d in shared_knowledge:
    chat_data.append({"text": d, "user_id":0})
embeddings.index(chat_data)

In [4]:
q = "What is the best type of food to eat in Chicago?"
mistral_q = f"<|prompter|>{q}</s><|assistant|>"

inputs = tokenizer(
    mistral_q,
    return_tensors="pt")

outputs = model.generate(
    **inputs, max_new_tokens=1000, use_cache=True, do_sample=True,
    temperature=0.2, top_p=0.95)

text = tokenizer.batch_decode(outputs)[0]

print(text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
user_id =1
query = f"select * from txtai where similar('{q}') AND (user_id={user_id} or user_id=0)"
results = embeddings.search(query)
print(results)
